### Initialization

In [2]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
spark.conf.set("spark.sql.shuffle.partitions", "4")

### Connection
Databricks notebooks are already connected to a cluster

In [5]:
# We already have a session called "spark"
print(type(spark))

# We also have a spark context called "sc"
print(type(sc))

<class 'pyspark.sql.session.SparkSession'>
<class '__main__.RemoteContext'>

### Pandas versus Spark DataFrames
Pandas DataFrames reside on the driver
Spark DataFrames are distributed on partitions across the cluster

In [7]:
sparkDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/databricks-datasets/definitive-guide/data/retail-data/by-day/*.csv")

sparkDataFrame.createOrReplaceTempView("retail_data")
staticSchema = sparkDataFrame.schema
print(staticSchema)

StructType(List(StructField(InvoiceNo,StringType,true),StructField(StockCode,StringType,true),StructField(Description,StringType,true),StructField(Quantity,IntegerType,true),StructField(InvoiceDate,TimestampType,true),StructField(UnitPrice,DoubleType,true),StructField(CustomerID,DoubleType,true),StructField(Country,StringType,true)))

In [8]:
sparkDataFrame.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 580538| 23084| RABBIT NIGHT LIGHT| 48|2011-12-05 08:38:00| 1.79| 14075.0|United Kingdom|
 580538| 23077| DOUGHNUT LIP GLOSS | 20|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom|
 580538| 22906|12 MESSAGE CARDS ...| 24|2011-12-05 08:38:00| 1.65| 14075.0|United Kingdom|
 580538| 21914|BLUE HARMONICA IN...| 24|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom|
 580538| 22467| GUMBALL COAT RACK| 6|2011-12-05 08:38:00| 2.55| 14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows

In [9]:
pandasDataFrame = sparkDataFrame.toPandas()
print(pandasDataFrame.shape)
pandasDataFrame.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,580538,23084,RABBIT NIGHT LIGHT,48,2011-12-05 08:38:00,1.79,14075.0,United Kingdom
1,580538,23077,DOUGHNUT LIP GLOSS,20,2011-12-05 08:38:00,1.25,14075.0,United Kingdom
2,580538,22906,12 MESSAGE CARDS WITH ENVELOPES,24,2011-12-05 08:38:00,1.65,14075.0,United Kingdom
3,580538,21914,BLUE HARMONICA IN BOX,24,2011-12-05 08:38:00,1.25,14075.0,United Kingdom
4,580538,22467,GUMBALL COAT RACK,6,2011-12-05 08:38:00,2.55,14075.0,United Kingdom


In [10]:
type(sparkDataFrame)

Out[7]: pyspark.sql.dataframe.DataFrame

In [11]:
sparkDataFrame.createOrReplaceTempView("purchases")

In [12]:
type(pandasDataFrame)

Out[9]: pandas.core.frame.DataFrame

In [13]:
pandasDataFrame.createOrReplaceTempView("purchases")

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2833190448324280> in <module> 
 ----> 1 pandasDataFrame . createOrReplaceTempView ( "purchases" ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/generic.py in __getattr__ (self, name) 
 5065 if self . _info_axis . _can_hold_identifiers_and_holds_name ( name ) : 
 5066 return self [ name ] 
 -> 5067 return object . __getattribute__ ( self , name ) 
 5068 
 5069 def __setattr__ ( self , name , value ) : 

 AttributeError : 'DataFrame' object has no attribute 'createOrReplaceTempView'

### Optimizing Execution Plans
The Catalyst optimizer takes advantage of lazy execution to optimize execution plans

In [15]:
# In the following cells, we will perform a series of transformations
# The full set of steps is shown here for clarity
# Note that the function withColumn() adds a new column
from pyspark.sql.functions import lit, expr

df0 = spark.range(0, 40, 2).toDF("c0")
df1 = df0.withColumn("c1", lit(20))
df2 = df1.withColumn("c2", expr("c1 - c0"))
df3 = df2.withColumn("c3", expr("c2 / 2"))
df4 = df3.where("c3 % 2 = 0")
df5 = df4.sort("c3")
df6 = df5.withColumn("c4", expr("c3 + 3")).withColumn("c5", expr("c4 * c4")).withColumn("c6", expr("c4 + c5"))
df7 = df6.sort("c6")
df7.show()

+---+---+---+----+----+-----+-----+
 c0| c1| c2| c3| c4| c5| c6|
+---+---+---+----+----+-----+-----+
 28| 20| -8|-4.0|-1.0| 1.0| 0.0|
 24| 20| -4|-2.0| 1.0| 1.0| 2.0|
 32| 20|-12|-6.0|-3.0| 9.0| 6.0|
 20| 20| 0| 0.0| 3.0| 9.0| 12.0|
 36| 20|-16|-8.0|-5.0| 25.0| 20.0|
 16| 20| 4| 2.0| 5.0| 25.0| 30.0|
 12| 20| 8| 4.0| 7.0| 49.0| 56.0|
 8| 20| 12| 6.0| 9.0| 81.0| 90.0|
 4| 20| 16| 8.0|11.0|121.0|132.0|
 0| 20| 20|10.0|13.0|169.0|182.0|
+---+---+---+----+----+-----+-----+

In [16]:
# The code for df3 creates a range, then adds columns
# The code has 4 steps, but Spark plan does it in 3 steps
df0 = spark.range(0, 40, 2).toDF("c0")
df1 = df0.withColumn("c1", lit(20))
df2 = df1.withColumn("c2", expr("c1 - c0"))
df3 = df2.withColumn("c3", expr("c2 / 2"))
df3.explain()

# The code for df4 adds a filter as a 5th step
# The Spark plan does the filter right after creating the range
# This is what predicate pushdown does
# It eliminates the need to calculate column values for the rows that eventually are filtered out
df4 = df3.where("c3 % 2 = 0")
df4.explain()


== Physical Plan ==
*(1) Project [c0#144L, 20 AS c1#146, c2#149L, (cast(c2#149L as double) / 2.0) AS c3#153]
+- *(1) Project [id#142L AS c0#144L, (20 - id#142L) AS c2#149L]
 +- *(1) Range (0, 40, step=2, splits=8)
== Physical Plan ==
*(1) Project [c0#144L, 20 AS c1#146, c2#149L, (cast(c2#149L as double) / 2.0) AS c3#153]
+- *(1) Project [id#142L AS c0#144L, (20 - id#142L) AS c2#149L]
 +- *(1) Filter (((cast((20 - id#142L) as double) / 2.0) % 2.0) = 0.0)
 +- *(1) Range (0, 40, step=2, splits=8)

In [17]:
# The code for df6 does a sort and adds more columns
df5 = df4.sort("c3")
df6 = df5.withColumn("c4", expr("c3 + 3")).withColumn("c5", expr("c4 * c4")).withColumn("c6", expr("c4 + c5"))
df6.explain()

# The code for df7 adds another sort
# The Spark plan only sorts the data once
df7 = df6.sort("c6")
df7.explain()


== Physical Plan ==
*(2) Project [c0#144L, 20 AS c1#146, c2#149L, c3#153, c4#158, c5#164, (c4#158 + c5#164) AS c6#171]
+- *(2) Project [c0#144L, c2#149L, c3#153, c4#158, (c4#158 * c4#158) AS c5#164]
 +- *(2) Project [c0#144L, c2#149L, c3#153, (c3#153 + 3.0) AS c4#158]
 +- *(2) Sort [c3#153 ASC NULLS FIRST], true, 0
 +- Exchange rangepartitioning(c3#153 ASC NULLS FIRST, 4), [id=#211]
 +- *(1) Project [c0#144L, c2#149L, (cast(c2#149L as double) / 2.0) AS c3#153]
 +- *(1) Project [id#142L AS c0#144L, (20 - id#142L) AS c2#149L]
 +- *(1) Filter (((cast((20 - id#142L) as double) / 2.0) % 2.0) = 0.0)
 +- *(1) Range (0, 40, step=2, splits=8)
== Physical Plan ==
Sort [c6#171 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(c6#171 ASC NULLS FIRST, 4), [id=#286]
 +- *(1) Project [c0#144L, 20 AS c1#146, c2#149L, c3#153, c4#158, c5#164, (c4#158 + c5#164) AS c6#171]
 +- *(1) Project [c0#144L, c2#149L, c3#153, c4#158, (c4#158 * c4#158) AS c5#164]
 +- *(1) Project [c0#144L, c2#149L, c3#153, (c3#153 + 3.0) AS c4#158]
 +- *(1) Project [c0#144L, c2#149L, (cast(c2#149L as double) / 2.0) AS c3#153]
 +- *(1) Project [id#142L AS c0#144L, (20 - id#142L) AS c2#149L]
 +- *(1) Filter (((cast((20 - id#142L) as double) / 2.0) % 2.0) = 0.0)
 +- *(1) Range (0, 40, step=2, splits=8)

### Spark SQL
Some computations are easier to express in SQL than with traditional Pandas-like code

In [19]:
spark.sql("""
  SELECT CustomerId, count(Quantity) AS num, SUM(UnitPrice * Quantity) AS total_cost
  FROM purchases
  WHERE CustomerId IN (SELECT CustomerId FROM purchases WHERE CustomerID < 14000)
  GROUP BY CustomerId
  HAVING count(Quantity) > 500
  """)\
  .collect()


Out[25]: [Row(CustomerId=13089.0, num=1857, total_cost=57385.87999999997),
 Row(CustomerId=13230.0, num=612, total_cost=2763.409999999999),
 Row(CustomerId=13137.0, num=705, total_cost=3605.080000000001),
 Row(CustomerId=12867.0, num=551, total_cost=3986.2199999999993),
 Row(CustomerId=12471.0, num=531, total_cost=18740.920000000002),
 Row(CustomerId=12921.0, num=741, total_cost=16389.739999999998),
 Row(CustomerId=13969.0, num=633, total_cost=8986.690000000002),
 Row(CustomerId=12681.0, num=646, total_cost=13677.590000000004),
 Row(CustomerId=12748.0, num=4642, total_cost=29072.09999999999),
 Row(CustomerId=13081.0, num=1061, total_cost=27964.479999999996),
 Row(CustomerId=13263.0, num=1677, total_cost=7454.0700000000015),
 Row(CustomerId=13668.0, num=501, total_cost=6216.070000000001),
 Row(CustomerId=13694.0, num=585, total_cost=62653.1),
 Row(CustomerId=12415.0, num=778, total_cost=123725.45000000001),
 Row(CustomerId=12682.0, num=525, total_cost=12288.219999999998),
 Row(CustomerId=13408.0, num=501, total_cost=27487.410000000003),
 Row(CustomerId=13098.0, num=605, total_cost=28658.879999999994)]

### Machine Learning
The following cells show a simple model being trained

In [21]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = sparkDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)


In [22]:
trainDataFrame = preppedDataFrame\
  .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
  .where("InvoiceDate >= '2011-07-01'")


In [23]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")


In [24]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")


In [25]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")


In [26]:
from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])


In [27]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)


In [28]:
transformedTraining = fittedPipeline.transform(trainDataFrame)


In [29]:
transformedTraining.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|day_of_week|day_of_week_index|day_of_week_encoded| features|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
 537226| 22811|SET OF 6 T-LIGHTS...| 6|2010-12-06 08:34:00| 2.95| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[2.95,...|
 537226| 21713|CITRONELLA CANDLE...| 8|2010-12-06 08:34:00| 2.1| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[2.1,8...|
 537226| 22927|GREEN GIANT GARDE...| 2|2010-12-06 08:34:00| 5.95| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[5.95,...|
 537226| 20802|SMALL GLASS SUNDA...| 6|2010-12-06 08:34:00| 1.65| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[1.65,...|
 537226| 22052|VINTAGE CARAVAN G...| 25|2010-12-06 08:34:00| 0.42| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[0.42,...|
 537226| 22705| WRAP GREEN PEARS | 25|2010-12-06 08:34:00| 0.42| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[0.42,...|
 537226| 20781|GOLD EAR MUFF HEA...| 2|2010-12-06 08:34:00| 5.49| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[5.49,...|
 537226| 22310|IVORY KNITTED MUG...| 6|2010-12-06 08:34:00| 1.65| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[1.65,...|
 537226| 22389|PAPERWEIGHT SAVE ...| 6|2010-12-06 08:34:00| 2.55| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[2.55,...|
 537227| 22941|CHRISTMAS LIGHTS ...| 2|2010-12-06 08:42:00| 8.5| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[8.5,2...|
 537227| 22696| WICKER WREATH LARGE| 6|2010-12-06 08:42:00| 1.95| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[1.95,...|
 537227| 22193|RED DINER WALL CLOCK| 2|2010-12-06 08:42:00| 8.5| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[8.5,2...|
 537227| 21212|PACK OF 72 RETROS...| 120|2010-12-06 08:42:00| 0.42| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[0.42,...|
 537227| 21977|PACK OF 60 PINK P...| 48|2010-12-06 08:42:00| 0.55| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[0.55,...|
 537227| 84991|60 TEATIME FAIRY ...| 48|2010-12-06 08:42:00| 0.55| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[0.55,...|
 537227| 21213|PACK OF 72 SKULL ...| 48|2010-12-06 08:42:00| 0.55| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[0.55,...|
 537227| 21080|SET/20 RED RETROS...| 12|2010-12-06 08:42:00| 0.85| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[0.85,...|
 537227| 22632|HAND WARMER RED R...| 48|2010-12-06 08:42:00| 2.1| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[2.1,4...|
 537227| 22315|200 RED + WHITE B...| 12|2010-12-06 08:42:00| 1.25| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[1.25,...|
 537227| 21232|STRAWBERRY CERAMI...| 12|2010-12-06 08:42:00| 1.25| 17677.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[1.25,...|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
only showing top 20 rows

In [30]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
  .setK(20)\
  .setSeed(1)


In [31]:
kmModel = kmeans.fit(transformedTraining)


In [32]:
summary = kmModel.summary
print("Cluster Sizes:")
print(summary.clusterSizes)
centers = kmModel.clusterCenters()
print("Cluster Centers:")
for center in centers:
  print(center)

Cluster Sizes:
[234864, 1, 1, 1, 8, 4, 2, 9025, 4, 144, 18, 6, 1414, 4, 11, 199, 5, 46, 99, 47]
Cluster Centers:
[3.95038286 5.4204348 0.19555147 0.19409105 0.1805811 0.17158866
 0.14926511]
[1.0400e+00 7.4215e+04 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00]
[ 1.0400e+00 -7.4215e+04 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00]
[ 3.897e+04 -1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 1.000e+00]
[ 5.43415e+03 -1.00000e+00 0.00000e+00 1.25000e-01 3.75000e-01
 0.00000e+00 5.00000e-01]
[ 7.5000e-03 -9.4045e+03 2.5000e-01 7.5000e-01 0.0000e+00 0.0000e+00
 0.0000e+00]
[ 1.6670865e+04 -1.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00 0.0000000e+00]
[ 1.51478227 62.18027701 0.21396122 0.21218837 0.13285319 0.21518006
 0.15102493]
[3.8500e-01 4.4435e+03 2.5000e-01 2.5000e-01 0.0000e+00 0.0000e+00
 5.0000e-01]
[1.01923611e+00 6.98694444e+02 2.77777778e-01 2.36111111e-01
 1.31944444e-01 2.15277778e-01 1.04166667e-01]
[ 1.90522556e+03 -1.11111111e-01 5.55555556e-02 1.66666667e-01
 3.88888889e-01 5.55555556e-02 3.33333333e-01]
[ 3.50000000e-01 -3.15500000e+03 0.00000000e+00 1.66666667e-01
 3.33333333e-01 5.00000000e-01 0.00000000e+00]
[1.35835926e+00 1.89192362e+02 2.43281471e-01 2.39038190e-01
 1.23762376e-01 2.05091938e-01 1.49222065e-01]
[ 1.3524695e+04 -5.0000000e-01 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00]
[6.54545455e-01 2.66545455e+03 2.72727273e-01 9.09090909e-02
 1.81818182e-01 1.81818182e-01 1.81818182e-01]
[1.31020101e+00 4.15376884e+02 3.06532663e-01 1.95979899e-01
 1.80904523e-01 1.90954774e-01 1.10552764e-01]
[ 7.385808e+03 -6.000000e-01 0.000000e+00 8.000000e-01 2.000000e-01
 0.000000e+00 0.000000e+00]
[ 4.19130435e-01 -1.05697826e+03 1.73913043e-01 2.39130435e-01
 2.39130435e-01 2.17391304e-01 1.30434783e-01]
[6.84855758e+02 9.09090909e-01 1.91919192e-01 2.32323232e-01
 2.82828283e-01 1.01010101e-01 1.91919192e-01]
[7.46382979e-01 1.38591489e+03 2.34042553e-01 1.91489362e-01
 1.06382979e-01 1.91489362e-01 2.12765957e-01]